### UNSEEN CANARI ###

CANARI HIST2 consists of a large ensemble of historical HadGEM3-GC31 runs between 1950-2014, with 40 members. The following variables are available:

* Daily temperature (for demand conversion)
* Hourly (!) 10m wind speed (for wind power conversion)

The temperature -> demand conversion is relatively simple. Convert to HDD/CDD and then use the regression relationships.

The 10m wind speed to wind power generation is more involved. Convert 10m wind speed to 100m wind speed through a power law conversion.

First we want to track down the daily temperature data for processing. To see whether we get similar answers using DePreSys for demand.

In [1]:
# Local imports
import os
import re
import sys
import glob

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import iris

# Specific imports
from tqdm import tqdm

In [2]:
# Set up the base canari dir
base_canari_dir = "/gws/nopw/j04/canari/shared/large-ensemble/priority/"

# /gws/nopw/j04/canari/shared/large-ensemble/priority/HIST2/3/ATM/yearly/1960/

period = "HIST2"
ensemble = "3"
domain = "ATM"
sample_year = "1960"
var_name = "m01s03i236_7"

# set up the years
years = np.arange(1950, 2014 + 1)

# sample fnam
sample_fnam = "cy537a_13_day__195407-195407.nc"

sample_fpath = "/gws/nopw/j04/canari/shared/large-ensemble/priority/HIST2/3/ATM/yearly/1960/cw345a_3_day_m01s03i236_7.nc"

In [3]:
# load in the sample file
sample_cube = iris.load_cube(sample_fpath)

/home/users/benhutch/.conda/envs/bens-conda-env2/lib/python3.11/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)
/home/users/benhutch/.conda/envs/bens-conda-env2/lib/python3.11/site-packages/iris/fileformats/cf.py:880: IrisCfMissingVarWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'm01s03i236_7'
  warnings.warn(


In [4]:
sample_cube.var_name

'm01s03i236_7'

In [5]:
# Function for adding realization metadata
# for r1i1p1f1, r2i1p1f1, etc.
def realization_metadata(cube, field, fpath):
    """Modify the cube's metadata to add a "realization" coordinate.

    A function which modifies the cube's metadata to add a "realization"
    (ensemble member) coordinate from the filename if one doesn't already exist
    in the cube.

    """
    # Add an ensemble member coordinate if one doesn't already exist.
    if not cube.coords("realization"):
        # The ensemble member is encoded in the filename as *_???.pp where ???
        # is the ensemble member.

        # # Regular expression pattern for the desired format
        # pattern = re.compile(r"(r\d+i\d+p\d+f\d+)")

        # Split the fpath by /
        fpath_split = fpath.split("/")

        # extract the member as the 9th element
        realization_number = fpath_split[9]

        # print the realization number
        print(realization_number)

        realization_coord = iris.coords.AuxCoord(
            np.int32(realization_number), "realization", units="1"
        )

        cube.add_aux_coord(realization_coord)

    return cube

In [6]:
# set up 10 files for the same year
members = np.arange(1, 11)

file_list = []

# loop over the members
for member in members:
    # set up the file path
    fpath = os.path.join(
        base_canari_dir, period, str(member), domain, "yearly", sample_year, f"*day_{var_name}.nc"
    )

    # glob the file path to check if it exists
    files = glob.glob(fpath)

    # if the file exists
    if len(files) == 1:
        # append the file to the list
        file_list.append(files[0])
    elif len(files) == 0:
        raise ValueError(f"No files found for {fpath}")
    else:
        raise ValueError(f"Multiple files found for {fpath}")


In [7]:
file_list

['/gws/nopw/j04/canari/shared/large-ensemble/priority/HIST2/1/ATM/yearly/1960/cv575a_1_day_m01s03i236_7.nc',
 '/gws/nopw/j04/canari/shared/large-ensemble/priority/HIST2/2/ATM/yearly/1960/cv625a_2_day_m01s03i236_7.nc',
 '/gws/nopw/j04/canari/shared/large-ensemble/priority/HIST2/3/ATM/yearly/1960/cw345a_3_day_m01s03i236_7.nc',
 '/gws/nopw/j04/canari/shared/large-ensemble/priority/HIST2/4/ATM/yearly/1960/cw356a_4_day_m01s03i236_7.nc',
 '/gws/nopw/j04/canari/shared/large-ensemble/priority/HIST2/5/ATM/yearly/1960/cv827a_5_day_m01s03i236_7.nc',
 '/gws/nopw/j04/canari/shared/large-ensemble/priority/HIST2/6/ATM/yearly/1960/cv976a_6_day_m01s03i236_7.nc',
 '/gws/nopw/j04/canari/shared/large-ensemble/priority/HIST2/7/ATM/yearly/1960/cz547a_7_day_m01s03i236_7.nc',
 '/gws/nopw/j04/canari/shared/large-ensemble/priority/HIST2/8/ATM/yearly/1960/cy436a_8_day_m01s03i236_7.nc',
 '/gws/nopw/j04/canari/shared/large-ensemble/priority/HIST2/9/ATM/yearly/1960/cw342a_9_day_m01s03i236_7.nc',
 '/gws/nopw/j04/can

In [8]:
# test loading the first file from the list
cube = iris.load_cube(file_list[0])

In [9]:
# set up the constraint
constraint = iris.Constraint(
    var_name,
    realization=lambda value: True,
)

# load in the cubes
cubes = iris.load(
    "/gws/nopw/j04/canari/shared/large-ensemble/priority/HIST2/*/ATM/yearly/1960/*_*_day_m01s03i236_7.nc",
    constraint,
    callback=realization_metadata,
)

In [15]:
print(cubes)

0: air_temperature / (K)               (time: 360; latitude: 324; longitude: 432)
1: air_temperature / (K)               (time: 360; latitude: 324; longitude: 432)
2: air_temperature / (K)               (time: 360; latitude: 324; longitude: 432)
3: air_temperature / (K)               (time: 360; latitude: 324; longitude: 432)
